In [5]:
import requests
from bs4 import BeautifulSoup
import csv

# URL to scrape
target_url = "https://www.netflix.com/in/title/80057281"
resp = requests.get(target_url)

# Check if the request was successful
if resp.status_code == 200:
    soup = BeautifulSoup(resp.text, 'html.parser')

    # Initialize dictionaries and lists to store data
    show_data = {}
    genres_list = []
    moods_list = []
    cast_list = []

    # Extract title
    title = soup.find("h1", {"class": "title-title"})
    if title:
        show_data["name"] = title.text

    # Extract seasons
    seasons = soup.find("span", {"class": "duration"})
    if seasons:
        show_data["seasons"] = seasons.text

    # Extract about
    about = soup.find("div", {"class": "hook-text"})
    if about:
        show_data["about"] = about.text

    # Extract genres
    genres = soup.find_all("span", {"class": "item-genres"})
    for genre in genres:
        genres_list.append({"genre": genre.text.replace(",", "")})

    # Extract moods
    moods = soup.find_all("span", {"class": "item-mood-tag"})
    for mood in moods:
        moods_list.append({"mood": mood.text.replace(",", "")})

    # Extract social links
    facebook = soup.find("a", {"data-uia": "social-link-facebook"})
    if facebook:
        show_data["facebook"] = facebook.get("href")

    twitter = soup.find("a", {"data-uia": "social-link-twitter"})
    if twitter:
        show_data["twitter"] = twitter.get("href")

    instagram = soup.find("a", {"data-uia": "social-link-instagram"})
    if instagram:
        show_data["instagram"] = instagram.get("href")

    # Extract cast
    cast = soup.find_all("span", {"class": "item-cast"})
    for member in cast:
        cast_list.append({"cast": member.text})

    # Extract image URL
    hero_image = soup.find("div", {"class": "hero-image-desktop"})
    if hero_image:
        image_style = hero_image.get("style")
        image_url = image_style.split("url(")[1].split(")")[0].replace("&quot;", "")
        show_data["image_url"] = image_url

    # Write to CSV
    csv_file = "netflix_data.csv"
    with open(csv_file, 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        
        # Write show data
        writer.writerow(["Show Information"])
        writer.writerow([f"Name: {show_data.get('name', '')}"])
        writer.writerow([f"Seasons: {show_data.get('seasons', '')}"])
        writer.writerow([f"About: {show_data.get('about', '')}"])
        writer.writerow([f"Facebook: {show_data.get('facebook', '')}"])
        writer.writerow([f"Twitter: {show_data.get('twitter', '')}"])
        writer.writerow([f"Instagram: {show_data.get('instagram', '')}"])
        writer.writerow([f"Image URL: {show_data.get('image_url', '')}"])  # Added image URL
        writer.writerow([])  # Blank row for separation

        # Write genres
        writer.writerow(["Genres"])
        for genre in genres_list:
            writer.writerow([genre.get("genre", "")])
        writer.writerow([])  # Blank row for separation

        # Write moods
        writer.writerow(["Moods"])
        for mood in moods_list:
            writer.writerow([mood.get("mood", "")])
        writer.writerow([])  # Blank row for separation

        # Write cast
        writer.writerow(["Cast"])
        for cast_member in cast_list:
            writer.writerow([cast_member.get("cast", "")])

    print(f"Data has been written to {csv_file}")
else:
    print("Failed to retrieve the webpage")


Data has been written to netflix_data.csv


In [6]:
import pandas as pd
from IPython.display import display, Image

# Load the CSV file
csv_file = "netflix_data.csv"
df = pd.read_csv(csv_file, header=None)

# Set a size for the image
image_size = (500, 500)  # Adjust size as needed

# Extract the image URL
image_row = df[df[0].str.contains("Image URL", na=False)]
if not image_row.empty:
    image_url = image_row.iloc[0, 0].split(': ')[1].strip('"').replace('""', '').strip()
else:
    image_url = None

# Display the image with the specified size
if image_url:
    display(Image(url=image_url, width=image_size[0], height=image_size[1]))

# Display show information
print("Show Information:")
for index in range(1, min(7, len(df))):  # Ensure we don't go out of bounds
    print(df.iloc[index, 0])

# Display genres
print("\nGenres:")
for index in range(9, min(13, len(df))):  # Ensure we don't go out of bounds
    print(df.iloc[index, 0].strip())

# Display moods
print("\nMoods:")
for index in range(14, min(17, len(df))):  # Ensure we don't go out of bounds
    print(df.iloc[index, 0].strip())

# Display cast
print("\nCast:")
for index in range(18, len(df)):  # Loop until the end of the DataFrame
    print(df.iloc[index, 0].strip())


Show Information:
Name: Stranger Things
Seasons: 4 Seasons
About: This nostalgic nod to '80s sci-fi and horror classics has earned dozens of Emmy nominations, including three for Outstanding Drama.
Facebook: https://www.facebook.com/StrangerThingsTV/
Twitter: https://twitter.com/stranger_things
Instagram: https://www.instagram.com/strangerthingstv

Genres:
Sci-Fi TV
Teen TV Shows
US TV Shows
Horror TV Serials

Moods:
Ominous
Nostalgic
Scary

Cast:
Winona Ryder
David Harbour
Millie Bobby Brown
Finn Wolfhard
Gaten Matarazzo
Caleb McLaughlin
Noah Schnapp
Sadie Sink
Natalia Dyer
Charlie Heaton
Joe Keery
Maya Hawke
Brett Gelman
Priah Ferguson
Dacre Montgomery
Cara Buono
Matthew Modine
Paul Reiser
Sean Astin
Shannon Purser
